In [2]:
import os

import numpy as np
from mayavi import mlab
from pyntcloud import PyntCloud
import re

/home/koondra/.local/lib/python3.10/site-packages/pyntcloud/utils/numba.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_count(xyz, indices, out):
/home/koondra/.local/lib/python3.10/site-packages/pyntcloud/utils/numba.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_sum(xyz, indices, N, out):
/home/koondra/.local/lib/python3.10/si

In [3]:
def visualize_points(points:np.ndarray, vis_num_tresh = 500000):

    if points.shape[0] > vis_num_tresh:
        print('too many points to visualize')
        return

    figure = mlab.figure(1, bgcolor=(1, 1, 1), size=(640, 480))
    nodes = mlab.points3d(points[:,0], points[:,1], points[:,2], points[:,3], scale_factor=0.05, scale_mode='none',colormap='jet')
    colorbar = mlab.colorbar()
    colorbar.label_text_property.color = (0, 0, 0)  
    mlab.show()
    
def visualize_points_with_feature(points:np.ndarray,feature:np.ndarray, vis_num_tresh = 500000):
    

    if points.shape[0] > vis_num_tresh:
        print('too many points to visualize')
        return
    
    if points.shape[0] != feature.shape[0]:
        print("shapes of points and features not matching")
        return

    figure = mlab.figure(1, bgcolor=(1, 1, 1), size=(640, 480))
    nodes = mlab.points3d(points[:,0], points[:,1], points[:,2], feature, scale_factor=0.05, scale_mode='none',colormap='jet')
    colorbar = mlab.colorbar()
    colorbar.label_text_property.color = (0, 0, 0)  
    mlab.show()
    
def load_pcd_to_numpy(pcd_filename:str):
    pcd_raw = PyntCloud.from_file(pcd_filename)
    pcd_numpy = pcd_raw.points.to_numpy()
    return pcd_numpy

In [62]:
file = '/home/koondra/sshfs_test/cylinder_test/test_55/sample_5.npz'

pc = np.load(file)['data']
print(pc.shape)

visualize_points_with_feature(pc, pc[:,-1])

(55000, 6)


mark the real files


In [70]:

sample_files = []
corr_files = []

annot_folder = '/home/koondra/temp_school/data/test_55/'
src_folder = '/home/koondra/temp_school/data/2clss_55rect_55kpts_5rect/pcd_adjusted'
target_folder = '/home/koondra/temp_school/data/test_55_merged/'
target_labels = '/home/koondra/temp_school/data/test_55_merged_labels/'

for filename in os.listdir(annot_folder):
    if filename.endswith('.npz') and filename.startswith('sample'):
        sample_files.append(os.path.join(annot_folder, filename))
        
for filename in os.listdir(src_folder):
    if filename.endswith('.pcd'):
        corr_files.append(os.path.join(src_folder, filename))
        
print(len(corr_files), len(sample_files))

868 868


In [72]:
assert len(corr_files) == len(sample_files)
from tqdm import tqdm

merged_paths = []

pbar = tqdm(total=len(sample_files))

for i in range(len(sample_files)):
    try:
        sample_pc = np.load(sample_files[i])['data']
        sample_labels = sample_pc[:, -1].reshape(-1, 1)
    except Exception as e:
        print(e)
        continue
    orig_pc = load_pcd_to_numpy(corr_files[i])
    
    merged_sample_pc = np.concatenate([orig_pc, sample_labels], axis=1)
    
    base_name = os.path.basename(corr_files[i])
    sample_name = base_name[:-4] + '.npz'
    sample_path = os.path.join(target_folder, sample_name)
    merged_paths.append(sample_path)

    label_sample_path = os.path.join(target_labels, sample_name)
    np.savez(label_sample_path,  data = sample_labels.reshape(-1))
    
    np.savez(sample_path, data = merged_sample_pc)
    pbar.update(1)
    
pbar.close()


 29%|██▉       | 256/868 [00:04<00:18, 32.39it/s]

Cannot load file containing pickled data when allow_pickle=False


100%|█████████▉| 867/868 [00:22<00:00, 37.85it/s]


In [68]:

import random
for i in range(10):
    
    idx = random.randint(0, len(merged_paths)-1)
    path = sample_files[idx]
    pc = np.load(path)['data']
    
    visualize_points_with_feature(pc, pc[:, -1] - pc[:, -2])

In [14]:
pc = np.load('/home/koondra/Downloads/segmentation(1).npy')
print(pc)

[[ 4.11883736e+00 -1.44940829e+00  3.74899179e-01  1.22000000e+02
   1.00000000e+00]
 [ 4.11883736e+00 -1.44940829e+00  3.74899179e-01  1.22000000e+02
   1.00000000e+00]
 [ 4.12217999e+00 -1.45096171e+00  3.73346090e-01  1.24000000e+02
   3.00000000e+00]
 ...
 [ 1.91648793e+02  1.95411934e+02  3.67153019e-01  2.54000000e+02
   1.20000000e+03]
 [ 1.91642697e+02  1.95431153e+02  6.45130873e-01  2.54000000e+02
   1.20000000e+03]
 [ 1.91648793e+02  1.95411934e+02  3.67153019e-01  2.54000000e+02
   1.20000000e+03]]


In [17]:
visualize_points(pc)